In [184]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse

In [196]:
# load all data sets
breitbart = pd.read_csv('data/breitbart_data.csv')
fox = pd.read_csv('data/fox_data.csv')
wt = pd.read_csv('data/wt_data.csv')
ap = pd.read_csv('data/ap_data.csv')
nbc = pd.read_csv('data/nbc_data.csv')
nyt = pd.read_csv('data/nyt_data.csv')
politico = pd.read_csv('data/politico_data.csv')
buzzfeed = pd.read_csv('data/buzzfeed_data.csv')

#### Make dates comparable

In [197]:
fox['date'] = [x.split('T')[0] for x in fox['date']]

In [198]:
wt['date'] = [x.replace(' -\n\t\t\t\n\t\t\t\tAssociated Press\n -    Updated: 6:23 a.m. on\n                        ', '') for x in wt['date']]
wt['date'] = [parse(x) for x in wt['date']]

In [199]:
ap['date'] = [x.split('T')[0] for x in ap['date']]

In [200]:
nbc = nbc.copy().dropna()
nbc['date'] = [parse(x) for x in nbc['date']]

In [201]:
buzzfeed['date'] = [x.replace('Posted on ', '').replace('Last updated on ', '') for x in buzzfeed['date']]
buzzfeed['date'] = [x.strip() for x in buzzfeed['date']]
buzzfeed['date'] = [x.split(',')[0:2] for x in buzzfeed['date']]
buzzfeed['date'] = [''.join(x) for x in buzzfeed['date']]
buzzfeed['date'] = [parse(x) for x in buzzfeed['date']]

#### Merge

In [210]:
full_data = pd.concat([
    breitbart,
    fox,
    wt,
    ap,
    nbc,
    nyt,
    politico,
    buzzfeed
])

#### Articles to sentences

In [211]:
text = full_data['article_text'].copy().str.split('.').apply(pd.Series, 1).stack()

In [212]:
text.index = text.index.droplevel(-1) 
text.name = 'article_text'

In [213]:
text.replace('', np.nan, inplace = True)
text.dropna(inplace = True)

In [214]:
sentence_data = full_data.copy()
del sentence_data['article_text']
sentence_data = sentence_data.join(test)

#### Find candidates

In [222]:
candidates = ['Biden', 'Bernie', 'Sanders', 'Warren', 'Trump',
             'Klobuchar', 'Buttigieg', 'Yang', 'Bloomberg',
             'Gabbard', 'Steyer', 'Deval', 'Bennet']